In [3]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re


# Bueno

In [ ]:

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])

def separar_y_limpiar(nombre):
    if pd.isna(nombre):
        return []
    separadores = [";", ",", "&", "Featuring", " feat.", "Feat.", " ft.", "/", " x "]
    for sep in separadores:
        nombre = nombre.replace(sep, "|")
    partes = nombre.split("|")
    return [re.sub(r'[^\w\s\-]', '', p.strip().strip('"').strip("'")) for p in partes if p.strip()]

todos_los_artistas = []
for nombre in df["raw"]:
    todos_los_artistas.extend(separar_y_limpiar(nombre))

artistas_unicos = sorted(set([a for a in todos_los_artistas if a]))
print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. CONSULTA SPARQL CON TODOS LOS CAMPOS
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?birth ?death ?countryLabel ?typeLabel ?genreLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P31 ?type. }}
      OPTIONAL {{ ?artist wdt:P136 ?genre. }}
      OPTIONAL {{ ?artist wdt:P569 ?birth. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}  # Género con P21
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "birth": row.get("birth", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "type": row.get("typeLabel", {}).get("value", ""),
                        "genre": row.get("genreLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")  # Valor por defecto si no hay género
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1.1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "type", "genre", "award", "birth", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)



🔄 Cargando y limpiando datos...
✅ Total artistas únicos: 30149
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 30180it [29:06, 17.28it/s]                            


✅ Archivo 'wikidata_artists.csv' creado correctamente.


In [5]:
# Calcular el punto de división (por la mitad en este caso)
halfway = len(df_result) // 2

# Guardar la primera parte del DataFrame
df_result.iloc[:halfway].to_csv("../data/wikidata_artists_part1.csv", index=False)
print("✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.")

# Guardar la segunda parte del DataFrame
df_result.iloc[halfway:].to_csv("../data/wikidata_artists_part2.csv", index=False)
print("✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.")

✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.
✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.
